## Wrapper methode (Backwards feature selection)

De methodes die gebruikt kunnen worden in het selecteren van features op nauwkeurigheid is forward- en backwards elimination (D.H. Vu, 2014). Bij foreward elimination wordt aan een leeg model telkens een feature toegevoegd, waardoor het beste model wordt gegenereerd. Met backwards elimination begint het model met alle features en door deze één voor één te verwijderen wordt het beste model gegenereerd. Een nadeel van foreward elimination is dat het geen variabelen kan bevatten die afhankelijk zijn van elkaar, met backwards elimination is dit wel mogelijk. Omdat uit het filtermodel blijk dat de features afhankelijk zijn van elkaar wordt er gebruik gemaakt van backwards elimination. 

Een manier om waarden te elimineren is door te kijken naar de significatie van de waarde in het model. Met de p-waarde kan de significantie berekend worden. Wanneer de p-waarde onder de 0.05 ligt is de feature significant in het model (D.H. Vu, 2014). 

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy import stats
from sklearn.feature_selection import f_regression

In [2]:
#ophalen van de dataset.
features = pd.read_csv('energie_features_inflatie.csv')
features.head()

,Datum,energie,wind_richting,zon_perc,zon_straling,dag_vocht,max_vocht,min_vocht,vector_wind,wind,...,min_temp_6m,max_temp_6m,zon_uren_6m,duur_neerslag_6m,dag_neerslag_6m,max_neerslag_6m,dag_luchtdruk_6m,max_luchtdruk_6m,min_luchtdruk_6m,CPI_energie_6m
0,2001-01,9267,141.483871,27.967742,241.870968,90.161290,96.580645,79.483871,3.403226,3.693548,...,11.641935,19.548387,3.954839,1.854839,3.151613,1.567742,1012.616129,1015.319355,1009.887097,54.71
1,2001-02,8266,181.642857,32.250000,483.571429,88.000000,97.464286,73.964286,3.050000,3.639286,...,11.593548,22.809677,6.825806,0.709677,1.380645,0.903226,1017.519355,1019.232258,1015.712903,54.71
2,2001-03,8962,142.580645,17.419355,570.774194,85.548387,95.451613,72.354839,3.448387,3.829032,...,12.110000,19.946667,3.846667,1.680000,2.280000,1.186667,1013.006667,1015.930000,1010.070000,54.68
3,2001-04,8156,226.400000,34.766667,1220.166667,78.933333,96.333333,56.133333,3.390000,3.926667,...,7.700000,14.954839,3.161290,3.006452,3.393548,1.358065,1010.248387,1014.483871,1005.770968,55.02
4,2001-05,8304,134.645161,56.967742,2028.741935,71.645161,93.548387,50.322581,3.619355,3.835484,...,5.393333,10.120000,2.030000,3.100000,3.916667,1.366667,1002.100000,1006.016667,997.723333,55.08


### Methode 1
De p-waarde is berekend met twee methode. In de eerste methode is de p-waarde met behulp van formules berekend (Find P-value (significance) in Scikit-learn Linear Regression, 2021). Met deze methode komen vijf significante features naar voren. Deze features zijn vervolgens getraind in een lineair regressiemodel. 

In [5]:
alle_features = features.drop(['Datum', 'energie'], axis=1)
X = alle_features.values 
y = features['energie'].values 

In [14]:
LR = LinearRegression()
LR.fit(X,y)

parameters = np.append(LR.intercept_,LR.coef_)
predictions = LR.predict(X)

new_X = np.append(np.ones((len(X),1)), X, axis=1)

MSE = (sum((y-predictions)**2))/(len(new_X)-len(new_X[0]))
v_b = MSE*(np.linalg.inv(np.dot(new_X.T,new_X)).diagonal())
s_b = np.sqrt(v_b)
t_b = parameters/ s_b

p_waarde =[2*(1-stats.t.cdf(np.abs(i),(len(new_X)-len(new_X[0])))) for i in t_b]
p_waarde = np.round(p_waarde,2)

features_pwaarde = list(zip(features.columns, (p_waarde<0.05) ))
features_pwaarde

[('Datum', False),
 ('energie', False),
 ('wind_richting', False),
 ('zon_perc', False),
 ('zon_straling', False),
 ('dag_vocht', False),
 ('max_vocht', False),
 ('min_vocht', False),
 ('vector_wind', False),
 ('wind', False),
 ('max_wind', False),
 ('min_wind', False),
 ('max_windstoot', False),
 ('gem_temp', False),
 ('min_temp', False),
 ('max_temp', False),
 ('zon_uren', False),
 ('duur_neerslag', False),
 ('dag_neerslag', False),
 ('max_neerslag', False),
 ('dag_luchtdruk', False),
 ('max_luchtdruk', False),
 ('min_luchtdruk', False),
 ('CPI_energie', False),
 ('wind_richting_1j', False),
 ('zon_perc_1j', False),
 ('zon_straling_1j', False),
 ('dag_vocht_1j', False),
 ('max_vocht_1j', False),
 ('min_vocht_1j', False),
 ('vector_wind_1j', False),
 ('wind_1j', False),
 ('max_wind_1j', False),
 ('min_wind_1j', False),
 ('max_windstoot_1j', False),
 ('gem_temp_1j', False),
 ('min_temp_1j', False),
 ('max_temp_1j', False),
 ('zon_uren_1j', False),
 ('duur_neerslag_1j', False),
 ('dag_n

In [15]:
X = features[['wind_richting_1m', 'max_temp_1m', 'wind_richting_6m', 'zon_perc_6m', 'max_temp_6m']].values
y = features['energie'].values 

In [16]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [17]:
LR = LinearRegression()
LR.fit(x_train,y_train)
y_prediction =  LR.predict(x_test)

In [19]:
score=r2_score(y_test,y_prediction)
print('r2 score = ',score)
print('RMSE = ',np.sqrt(mean_squared_error(y_test,y_prediction)))

r2 score =  0.43056474581951587
RMSE =  454.6439192590668


### Methode 2
De p-waarde is berekend met twee methode. De tweede methode berekend de p-waarde met de f_regression formule die in sklearn gedefinieerd is (sklearn.feature_selection.f_regression, z.d.). Hierin komen 31 features naar voren die significant zijn wanneer de features constant zijn. Doordat de features constant zijn en geen perfecte correlatie hebben met de energie worden de waarden die False zijn meegenomen in het lineaire machine learning model.

In [21]:
alle_features = features.drop(['Datum', 'energie'], axis=1)
X = alle_features.values 
y = features['energie'].values

f_reg = f_regression(X,y, force_finite=False)
f_reg = np.round(f_reg,2)

feature_f_reg = list(zip(features.columns, (f_reg<0.05)))
feature_f_reg

[('Datum',
  array([False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
          True, False, False, False, False, False, False, False, False,
         False, False, False, False,  True, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False,  True, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False])),
 ('energie',
  array([False,  True,  True,  True, False,  True,  True,  True,  True,
          True

In [22]:
features_energie = features[['zon_straling', 'zon_uren', 'dag_neerslag', 'max_neerslag', 'min_luchtdruk',
                                  'zon_straling_1j', 'zon_uren_1j', 'dag_neerslag_1j', 'max_neerslag_1j', 'dag_luchtdruk_1j', 
                                  'min_luchtdruk_1j', 'vector_wind_1m', 'max_wind_1m', 'zon_uren_1m', 'duur_neerslag_1m', 'dag_neerslag_1m',
                                  'max_neerslag_1m', 'dag_luchtdruk_1m', 'min_luchtdruk_1m', 'CPI_energie_1m', 'wind_richting_3m', 
                                  'dag_vocht_3m', 'min_temp_3m', 'max_temp_3m', 'dag_neerslag_3m', 'max_neerslag_3m', 'dag_luchtdruk_3m', 
                                  'zon_straling_6m', 'zon_uren_6m', 'dag_neerslag_6m', 'max_neerslag_6m']]
X = features_energie.values
y = features['energie'].values 

In [23]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [24]:
LR = LinearRegression()
LR.fit(x_train,y_train)
y_prediction =  LR.predict(x_test)

In [25]:
score=r2_score(y_test,y_prediction)
print('r2 score = ',score)
print('RMSE = ',np.sqrt(mean_squared_error(y_test,y_prediction)))

r2 score =  0.7220958290629154
RMSE =  317.61188388957856


## Bronnenlijst
D.H. Vu, K. M. (2014, December 23). A variance inflation factor and backward elimination based robust regression model for forecasting monthly electricity demand using climatic variables. Opgehaald van Science Direct: https://www.sciencedirect.com/science/article/pii/S0306261914012604 <br>
Find P-value (significance) in Scikit-learn Linear Regression. (2021, Oktober 27). Opgehaald van Data Courses: https://www.datacourses.com/find-p-value-significance-in-scikit-learn-3810/ <br>
sklearn.feature_selection.f_regression. (sd). Opgehaald van scikit-learn: https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_regression.html